# Widgets

In this example we will use the [iPyWidgets Jupyter](https://ipywidgets.readthedocs.io) extension to interactively tune simulation or model parameters.
The user will see immediatly the impact of parameter changes as the simulation results are continously updated.

This example uses slider widgets to adjust parameters:

In [ ]:
import dpsim
import math, cmath
import time
import os

import matplotlib as plt
import villas.dataprocessing.plottools as pt
import villas.dataprocessing.readtools as rt

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

We start by defining our system topology:

In [ ]:
gnd = dpsim.dp.Node.GND()
n1 =  dpsim.dp.Node("n1")
n2 =  dpsim.dp.Node("n2")

cs = dpsim.dp.ph1.CurrentSource("cs", [gnd, n1])
r1 = dpsim.dp.ph1.Resistor("r_1", [n1, gnd])
c1 = dpsim.dp.ph1.Capacitor("c_1", [n1, n2])
l1 = dpsim.dp.ph1.Inductor("l_1", [n2, gnd])
r2 = dpsim.dp.ph1.Resistor("r_2", [n2, gnd])

sys = dpsim.SystemTopology(50, [gnd, n1, n2], [cs, r1, c1, l1, r2])
sim = dpsim.Simulation("Widgets", sys, duration=0.1)
    
logger = dpsim.Logger("Widgets")
logger.log_attribute(n1, "v") # v1
logger.log_attribute(n2, "v") # v2
logger.log_attribute(cs, "i_intf") # i12
logger.log_attribute(c1, "i_intf") # i34
sim.add_logger(logger)

Next, we define the simulation callback. This function will be executed on every parameter change.
It will re-run the simulation and plot the updated results.

In [ ]:
def simulate(dt, dur, pha, mag, r1_val, r2_val, l_val, c_val):
    sim.reset()
    
    sim.timestep = dt
    sim.final_time = dur
    cs.I_ref = cmath.rect(mag, pha)
    r1.R = r1_val
    c1.C = c_val
    l1.L = l_val * 1e-3
    r2.R = r2_val * 1e-3
    
    sim.start()
    while sim.state != 9:
        time.sleep(0.001)
        
    results = rt.read_timeseries_dpsim('logs/Widgets.csv')
    for l in [ 'n1.v', 'n2.v', 'cs.i_intf', 'c_1.i_intf' ]:
        emt = results[l].frequency_shift(50)
        pt.plot_timeseries(1, emt)

Set the `cu` variable to `True` for a continous redrawing of the plot while dragging the sliders.

In [ ]:
cu = False

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

output = interactive(simulate,
    dt      = widgets.FloatText(description="Timestep [s]", value=1e-3, min=1e-3, max=1),
    dur     = widgets.FloatText(description="Duration [s]", value=0.5, min=0.001, max=4),
    pha     = widgets.FloatSlider(description="Phase [rad]", min=-math.pi, max=math.pi, continuous_update=cu),
    mag     = widgets.FloatSlider(description="Magnitude [V]", value=10, min=0, max=100, continuous_update=cu),
    r1_val  = widgets.FloatSlider(description="Resistance [Ohm]", value=1, min=0.1, max=10, continuous_update=cu),
    r2_val  = widgets.FloatSlider(description="Resistance [Ohm]", value=1, min=0.1, max=10, continuous_update=cu),
    l_val   = widgets.FloatSlider(description="Inductance [H]", value=1, min=1, max=10, continuous_update=cu),
    c_val   = widgets.FloatSlider(description="Capactance [F]", value=1, min=1, max=10, continuous_update=cu)
)

last = output.children[-1]
last.layout.height = '400px'
output